###### [ 한글 데이터셋 RNN 모델 만들기 ] <hr>
- Dataset: NSMC from Korpora

Data loading:

In [99]:
# import modules

from  Korpora import Korpora
import pandas as pd
import numpy as np
import torch 
import torch.nn as nn

In [100]:
# data loading

corpus1 = Korpora.load('nsmc')
corpusDF = pd.DataFrame(corpus1.test)

corpusDF.head()


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\kdp\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\kdp\K

,text,label
0,굳 ㅋ,1
1,GDNTOPCLASSINTHECLUB,0
2,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [101]:
## 학습용 / 테스트용 데이터 분리하기

trainDF = corpusDF.sample(frac=0.9, random_state=42)
trainDF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45000 entries, 33553 to 6838
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    45000 non-null  object
 1   label   45000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.0+ MB


In [102]:
testDF = corpusDF.drop(labels=trainDF.index)
testDF.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5000 entries, 9 to 49997
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    5000 non-null   object
 1   label   5000 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 117.2+ KB


2. 단어사전 생성하기 <hr>

 - 토큰화 진행하기 -> 형태소 분석기 선택, 분할
 - 단어사전 만들기

2 - 1. 토큰화 진행: 문장을 단어로 쪼개기.

In [103]:
# module importing

from konlpy.tag import Okt

#tokenizer instance
okt = Okt()


# 자주 쓰는 이상자 정의
UNK = '<UNK>'
PAD = '<PAD>'

#문장 -> 단어

In [104]:
# for text in trainDF.text:
#     print(okt.morphs(text, stem=True)) # stem=True -> 단어의 어근으로 변형하여 추출하기.
#     break

train_tokens = [okt.morphs(text, stem=True) for text in trainDF.text]
test_tokens = [okt.morphs(text, stem=True) for text in testDF.text]

In [105]:
print(f'[TRAIN TOKENS] {len(train_tokens)}\n[TEST TOKENS] {len(test_tokens)}')
print(f'[TRAIN TOKENS[0]] {len(train_tokens[0])}\n[TEST TOKENS[0]] {len(test_tokens[0])}')

[TRAIN TOKENS] 45000
[TEST TOKENS] 5000
[TRAIN TOKENS[0]] 19
[TEST TOKENS[0]] 18


In [106]:
from collections import Counter

In [107]:
# 단어사전 생성하기
def build_vocab(corpus, vocab_size, special_tokens):
    counter = Counter() # 단어 세는 애

    # 단어 / 토큰에 대한 빈도 수 계산
    for tokens in corpus:
        counter.update(tokens)

    # 단어 / 어휘 사전 생성하기
    vocab = special_tokens

    # 빈도수가 높은 단어부터 단어사전에 추가하기. 
    for token, count in counter.most_common(vocab_size):
        vocab.append(token)
    return vocab

In [108]:
# tokenizer = okt
# train_tokens = [tokenizer.morphs(review) for review in trainDF.text]
# test_tokens = [tokenizer.morphs(review)for review in testDF.text]

In [109]:
vocab = build_vocab(corpus = train_tokens, vocab_size = 3000, special_tokens= ['<PAD>', '<UNK>'])

In [110]:
print(f'[VOCAB SIZE] {len(vocab)}\n{vocab[:30]}')

[VOCAB SIZE] 3002
['<PAD>', '<UNK>', '.', '이', '영화', '보다', '하다', '의', '..', '에', '가', '...', '을', '도', '들', ',', '는', '를', '은', '없다', '이다', '있다', '좋다', '?', '너무', '다', '정말', '한', '되다', '재밌다']


2 - 3. 인코딩 - 디코딩 인덱싱

In [111]:
# 인코딩 : 문자 -> 숫자
encode = {token: idx for idx, token in enumerate(vocab)}

# 디코딩 : 숫자 -> 문자
decode = {idx: token for idx, token in enumerate(vocab)}


In [112]:
# 리뷰의 문자를 정수로 변환 / 단어사전에 없는 문자 처리

UNK_ID = encode.get('<UNK>')
train_id = [[encode.get(token, UNK_ID) for token in text]for text in train_tokens]
test_id = [[encode.get(token,UNK_ID) for token in text] for text in test_tokens]

## 토큰 정수화

3 - 2. 데이터 구성 단어 수 맞추기 : 패딩
    - 단어 수 선정하기
    - 선정된 단어 수에 맞게 데이터 조잘: 길면 자르고 짧으면 채운다

In [113]:
# 패딩 처리 함수
## -sentences : 토큰화된 문장 데이터
## -max_length : 최대 문장 길이= 문장 1개를 구성하는 단어의 수
## -pad : 패딩 처리시 추가될 문자 값
## - start : 패딩 시 처리 방향 (기본 : R(오른쪽) 자르기 / 추가하기)

def pad_sequences(sentences, max_length, pad, start = 'R'):
    result = []
    for sen in sentences:
        sen = sen[:max_length] if start == 'R' else sen[:-1*max_length] #start 매개변수가 R이면 오른쪽에서 잘라내기
        padd_sen = sen + [pad]*(max_length - len(sen)) if start =='R' else ([pad]*(max_length - len(sen)) + sen) # start 매개변수가 R이면 오른쪽부터 패딩 넣기
        result.append(padd_sen)

    return result

In [114]:
# 학습용, 테스트용 데이터 패딩 처리
PAD_ID = encode.get('<PAD>')
MAX_LENGTH = 32

train_id = pad_sequences(train_id, MAX_LENGTH, PAD_ID)
test_id = pad_sequences(test_id, MAX_LENGTH, PAD_ID)

In [115]:
print(f'[TRAIN ID] {len(train_id[0])}\n[TEST ID] {len(test_id[0])}') # 0번 원소를 봐야 됨. 

[TRAIN ID] 32
[TEST ID] 32


4. 학습 준비 <hr>
- 데이터로더 준비하기
- 학습용 / 테스트용 함수
- 모델 클래스 생성하기
- 학습 관련 변수 설정: DEVICE, OPTIMIZER, MODEL INSTANCE, EPOCHS, BATCH_SIZE, LOSS FUNCTION, 

In [116]:
# 4 - 1. Dataloader 만들기

from torch.utils.data import DataLoader, TensorDataset

In [144]:
#dataset 생성: List -> TensorDataset
# 학습용 데이터셋
dataTS = torch.LongTensor(train_id)
labelTS = torch.FloatTensor(trainDF.label.values)


print(dataTS.shape, labelTS.shape)
trainDS = TensorDataset(dataTS, labelTS)


# 테스트용 데이터셋
testdataTS = torch.LongTensor(test_id)
testlabelTS = torch.FloatTensor(testDF.label.values)
testDS = TensorDataset(testdataTS, testlabelTS)
print(testdataTS.shape, testlabelTS.shape)



torch.Size([45000, 32]) torch.Size([45000])
torch.Size([5000, 32]) torch.Size([5000])


In [145]:
# 데이터로더 생성
BATCH_SIZE = 32

trainDL = DataLoader(trainDS, batch_size = BATCH_SIZE, shuffle = True)
testDL = DataLoader(testDS, batch_size = BATCH_SIZE, shuffle = True)


4 - 2. 모델 클래스 정의하기
- 입력층 : Embedding Layer
- 은닉층:  RNN / LSTM layer, Dropout layer
- 출력층 : Linear layer 

In [146]:
from torch import nn

class CLASSIFIER(nn.Module):
    def __init__(self, vocab_size,hidden_dim, embedding_dim, n_layers, dropout=0.5, bidirectional = True, model_type = 'lstm'):
        super().__init__()

        # 임베딩 설정
        self.embedding = nn.Embedding(
            num_embeddings = vocab_size,
            embedding_dim = embedding_dim,
            padding_idx=0)
        
        # 모델 지정이 lstm이라면
        if model_type == 'lstm':
            self.model = nn.LSTM(
                input_size = embedding_dim,
                hidden_size=hidden_dim,
                num_layers=n_layers,
                bidirectional= bidirectional,
                dropout = dropout,
                batch_first = True)
            
        # 모델 지정이 rnn이라면
        elif model_type == 'rnn':
            self.model = nn.RNN(
                input_size = embedding_dim,
                hidden_size = hidden_dim,
                num_layers = n_layers,
                biderectional = bidirectional,
                dropout = dropout,
                batch_first= True)
            
            #bidirectional(양방향 진행) 파라미터가 true라면
        if bidirectional == True:
            self.classifier = nn.Linear(hidden_dim * 2,1) # (양방향이니까) 리니어층을 두 배로 만들어주고 출력은 하나
        else: 
            self.classifier = nn.Linear(hidden_dim, 1) #false라면
            
        self.dropout = nn.Dropout(dropout) # 드롭아웃: 노드를 랜덤하게 비활성화시킵니다. 

    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        output, _ = self.model(embeddings)
        last_output = output[:,-1,:]
        last_output = self.dropout(last_output) # dropout은 연산을 안합니다. 비활성화를 시킵니다.
        logits = self.classifier(last_output)
        return logits ## 이진분류

In [147]:
## 손실함수 / 최적화함수 정의하기

from torch import optim
VOCAB_SIZE = len(encode)
HIDDEN_DIM = 64
EMBEDDING_DIM = 128
N_LAYERS = 2

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
S_CLASSIFIER= CLASSIFIER(vocab_size=VOCAB_SIZE, hidden_dim=HIDDEN_DIM, embedding_dim=EMBEDDING_DIM, n_layers=N_LAYERS).to(DEVICE)
criterion = nn.BCEWithLogitsLoss().to(DEVICE)
optimizer = optim.Adam(S_CLASSIFIER.parameters(), lr=0.001)


In [148]:
S_CLASSIFIER.classifier

Linear(in_features=128, out_features=1, bias=True)

In [149]:
# 모델 학습 및 테스트

def train(model, dataset, criterion, optimizer, device, interval):
    model.train()
    losses = list()

    for step,(inputs, labels) in enumerate(dataset):
        inputs = inputs.to(device)
        labels = labels.to(device).unsqueeze(1)

        logits = model(inputs)
        loss = criterion(logits, labels)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % interval == 0:
            print(f'train loss {step}: {np.mean(losses)}')

def test(model, dataset, criterion, device):
    model.eval()
    losses = list()
    corrects = list()

    for step, (inputs, labels) in enumerate(dataset):
        inputs = inputs.to(device)
        labels = labels.to(device).unsqueeze(1)

        logits = model(inputs)
        loss = criterion(logits, labels)
        losses.append(loss.item())

        yhat = torch.sigmoid(logits)>.5
        corrects.extend(torch.eq(yhat,labels).cpu().tolist())

        print(f'Val_Loss : {np.mean(losses)}, val_acc:{np.mean(corrects)}')
    

epoch = 5
interval = 500

In [150]:
for ep in range(epoch):
    train(S_CLASSIFIER, trainDL, criterion, optimizer, DEVICE, interval)
    test(S_CLASSIFIER, testDL, criterion, DEVICE)

train loss 0: 0.682779848575592
train loss 500: 0.6863231033146262
train loss 1000: 0.6770756258354796
Val_Loss : 0.6470910310745239, val_acc:0.65625
Val_Loss : 0.5949089825153351, val_acc:0.6875
Val_Loss : 0.5449325640996298, val_acc:0.71875
Val_Loss : 0.5597177594900131, val_acc:0.71875
Val_Loss : 0.5629166007041931, val_acc:0.7125
Val_Loss : 0.5427575955788294, val_acc:0.734375
Val_Loss : 0.5576777415616172, val_acc:0.7098214285714286
Val_Loss : 0.5651804767549038, val_acc:0.7109375
Val_Loss : 0.5611190232965682, val_acc:0.7118055555555556
Val_Loss : 0.5776222854852676, val_acc:0.696875
Val_Loss : 0.5810670337893746, val_acc:0.6903409090909091
Val_Loss : 0.5961141362786293, val_acc:0.6692708333333334
Val_Loss : 0.5909106387541845, val_acc:0.6778846153846154
Val_Loss : 0.5990898204701287, val_acc:0.6674107142857143
Val_Loss : 0.6008637448151907, val_acc:0.6708333333333333
Val_Loss : 0.59946727193892, val_acc:0.671875
Val_Loss : 0.6016421405708089, val_acc:0.6672794117647058
Val_Loss 